<a href="https://colab.research.google.com/github/AndreaBertoglio/MLDM/blob/master/Autosklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install swig -y
!pip install Cython numpy

!pip install auto-sklearn

In [41]:
import pip
import sys
#if not 'sklearn' in sys.modules.keys():
#    pip.main(['install', 'sklearn'])
#if not 'kaggle' in sys.modules.keys():
#    pip.main(['install', 'kaggle'])
import random

print("Random number with seed 2020")
# first call
random.seed(2020)

Random number with seed 2020


In [37]:
import numpy as np
import pandas as pd

import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics

import autosklearn.classification
import autosklearn.metrics

In [3]:
    def predict_proba(self, X, batch_size=None, n_jobs=1):
        # return self._automl.predict(X, batch_size=batch_size, n_jobs=n_jobs)
        return super().predict(X, batch_size=batch_size, n_jobs=n_jobs)

In [4]:
def accuracy(solution, prediction):
    # function defining accuracy
    return np.mean(solution == prediction)

In [5]:
def accuracy_wk(solution, prediction, dummy):
    # function defining accuracy and accepting an additional argument
    assert dummy is None
    return np.mean(solution == prediction)

In [38]:
import numpy as np
import pandas as pd
import graphviz

from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import matplotlib.pyplot as plt

# Read the data from GitHub
train = pd.read_csv('https://raw.githubusercontent.com/serivan/mldmlab/master/Datasets/Kaggle2020/train.csv')
train["Quality"] = np.where(train["Quality"].str.contains("Good"), 1, 0)


In [7]:
# pull data into target (y) and predictors (X)
#La y è la classe, cioè la Quality
train_y = train.Quality
#seleziono colonne di interesse, non metto l'ID perchè non mi interessa
predictor_cols = ['fixed.acidity','volatile.acidity','citric.acid','residual.sugar','chlorides','free.sulfur.dioxide','total.sulfur.dioxide','density','pH','sulphates','alcohol']

# La x sono gli attributi
train_X = train[predictor_cols]

# Sostituisce i missing values con la media e lo applica alle x
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = imp.fit(train_X)


# Impute our data, then train
train_X_imp = imp.transform(train_X)


In [13]:
#Uso l'80% dei dati per train e il restante 20% per test
X_train, X_test, y_train, y_test = train_test_split(train_X_imp, train_y, train_size = 0.8, random_state = 1)

In [9]:
# Print a list of available metrics
print("Available CLASSIFICATION metrics autosklearn.metrics.*:")
print("\t*" + "\n\t*".join(autosklearn.metrics.CLASSIFICATION_METRICS))

#print("Available REGRESSION autosklearn.metrics.*:")
#print("\t*" + "\n\t*".join(autosklearn.metrics.REGRESSION_METRICS))


Available CLASSIFICATION metrics autosklearn.metrics.*:
	*accuracy
	*balanced_accuracy
	*roc_auc
	*average_precision
	*log_loss
	*precision
	*precision_macro
	*precision_micro
	*precision_samples
	*precision_weighted
	*recall
	*recall_macro
	*recall_micro
	*recall_samples
	*recall_weighted
	*f1
	*f1_macro
	*f1_micro
	*f1_samples
	*f1_weighted


### Define accuracy metrics

In [29]:
# Define the model and the accuracy metrics
print("#"*80)
accuracy_scorer = autosklearn.metrics.make_scorer(name="accu",
                                                  score_func=accuracy,
                                                  greater_is_better=True,
                                                  needs_proba=False,
                                                  needs_threshold=False,
                                                  dummy=None)
cls = autosklearn.classification.\
    AutoSklearnClassifier(time_left_for_this_task=60,
                          per_run_time_limit=30, seed=1)
cls.fit(X_train, y_train) #, metric=accuracy_scorer)


################################################################################


TypeError: ignored

In [ ]:
predictions = cls.predict(X_test)
print("Confusion Matrix:")
print(confusion_matrix(y_train, predictions))

print("Classification Report:")
print(classification_report(y_train, predictions))

Confusion Matrix:
[[ 267  938]
 [ 225 2059]]
Classification Report:
              precision    recall  f1-score   support

           0       0.54      0.22      0.31      1205
           1       0.69      0.90      0.78      2284

    accuracy                           0.67      3489
   macro avg       0.61      0.56      0.55      3489
weighted avg       0.64      0.67      0.62      3489



In [22]:
print("Accuracy score {:g} using {:s}".
      format(sklearn.metrics.accuracy_score(y_test, predictions),
             "accu_add"))

Accuracy score 0.772206 using accu_add


### AutoSklearnClassifier

In [32]:
import autosklearn.classification
auto = autosklearn.classification.AutoSklearnClassifier()
auto.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


AutoSklearnClassifier(delete_output_folder_after_terminate=True,
                      delete_tmp_folder_after_terminate=True,
                      disable_evaluator_output=False,
                      ensemble_memory_limit=1024, ensemble_nbest=50,
                      ensemble_size=50, exclude_estimators=None,
                      exclude_preprocessors=None, get_smac_object_callback=None,
                      include_estimators=None, include_preprocessors=None,
                      initial_configurations_via_metalearning=25,
                      logging_config=None, max_models_on_disc=50,
                      metadata_directory=None, metric=None,
                      ml_memory_limit=3072, n_jobs=None, output_folder=None,
                      per_run_time_limit=360, resampling_strategy='holdout',
                      resampling_strategy_arguments=None, seed=1,
                      shared_mode=False, smac_scenario_args=None,
                      time_left_for_this_task=3600,

In [35]:
predictions = auto.predict(X_test)
print("Confusion Matrix:")
print(confusion_matrix(y_train, predictions))

print("Classification Report:")
print(classification_report(y_train, predictions))

Confusion Matrix:


ValueError: ignored

### Generate Submission

In [45]:
def generateSubmission(myModel, submissionFile, description):
    # Read the test data
    test = pd.read_csv('https://raw.githubusercontent.com/serivan/mldmlab/master/Datasets/Kaggle2020/test.csv')


    # Treat the test data in the same way as training data. In this case, pull same columns.
    test_X = test[predictor_cols]

    # Impute each test item, then predict
    test_X_imp = imp.transform(test_X)
    
    # Use the model to make predictions
    predicted_q = myModel.predict(test_X_imp)
    # We will look at the predicted Qualities to ensure we have something sensible.
    print(predicted_q)
    
    #submission file
    my_submission = pd.DataFrame({'Id': test.Id, 'Quality': predicted_q})
    # you could use any filename. We choose submission here
    my_submission.to_csv(submissionFile, index=False)
    
    #Submit authomatically; kaggle API authentication needed
    #!kaggle competitions submit -c mldm-classification-competition-2020 -f {submissionFile} -m '{description}'

In [ ]:
# generate a submission file for clf model
generateSubmission(clf,'AutosklearnClfTEST.csv', "Autosklearn di base con metrica personalizzata")

In [ ]:
# generate a submission file for auto model
generateSubmission(auto,'AutosklearnTEST.csv', "Autosklearn di base")